<a href="https://colab.research.google.com/github/BahadirAhmedov/01-todo-list/blob/main/labs/%D0%9B%D0%A01_%D0%9C%D0%B0%D0%BD%D0%B8%D0%BF%D1%83%D0%BB%D1%8F%D1%86%D0%B8%D0%B8_%D1%81_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%87%D0%BD%D1%8B%D0%BC%D0%B8_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Постановка задачи


**Цель работы:** изучение основных способов обработки табличных данных и создания запросов к ним с помощью модуля `pandas`.

**Краткое описание:** в лабораторной работе представлено 8 заданий на сортировку, фильтрацию и группировку данных методами модуля `pandas`. Первые 4 задания являются обязательными. Из оставшихся 4 необходимо выполнить любые два на выбор обучающегося.

# Данные


В папке [Data/lab1](https://drive.google.com/drive/folders/1RxzrWKQC5zqStrKE_3TB9C3Hqzqf-7aJ?usp=share_link) расположено несколько таблиц с обезличенными транзакционными банковскими данными. Все задания лабораторной работы необходимо выполнять по этим данным.

### Таблица ```transactions.csv```
##### **Описание**
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### **Формат данных**

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### **Описание полей**

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком:
  - ```+``` — начисление средств клиенту (приходная транзакция);
  - ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала.


 ### Таблица ```gender_train.csv```

##### **Описание**
Таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### **Формат данных**
```
customer_id,gender
111111,0
111112,1
...
```

##### **Описание полей**
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента.

### Таблица ```tr_mcc_codes.csv```

##### **Описание**
Таблица содержит описание mcc-кодов транзакций.

##### **Формат данных**
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### **Описание полей**
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

### Таблица ```tr_types.csv```

##### **Описание**
Таблица содержит описание типов транзакций.

##### **Формат данных**
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### **Описание полей**
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции.

# Практические задания

In [15]:
import pandas as pd
import numpy as np

## Подготовка

Загрузите и прочитайте данные из таблиц в переменные **tr_mcc_codes, tr_types, transactions** и **gender_train** из одноименных таблиц.

Выведите размеры таблиц и случайные строки из них.


In [61]:
tr_mcc_codes = pd.read_csv('/content/sample_data/tr_mcc_codes.csv', sep=';')
tr_types = pd.read_csv('/content/sample_data/tr_types.csv', sep=';')
transactions = pd.read_csv('/content/sample_data/transactions.csv', sep=',')
gender_train = pd.read_csv('/content/sample_data/gender_train.csv', sep=',')

print("Размеры таблиц:")
print("tr_mcc_codes:", tr_mcc_codes.shape)
print("tr_types:", tr_types.shape)
print("transactions:", transactions.shape)
print("gender_train:", gender_train.shape)

print("\nСлучайные строки из таблиц:")
display(tr_mcc_codes.sample(3))
display(tr_types.sample(3))
display(transactions.sample(3))
display(gender_train.sample(3))

Размеры таблиц:
tr_mcc_codes: (184, 2)
tr_types: (155, 2)
transactions: (6849346, 6)
gender_train: (8400, 2)

Случайные строки из таблиц:


,mcc_code,mcc_description
7,3501,"Жилье — отели, мотели, курорты"
138,7298,Центры здоровья
130,6536,Денежные переводы MasterCard MoneySend


,tr_type,tr_description
27,4210,Плата за получение наличных в АТМ. Зарубеж.
131,2400,Перевод средств с карты на счет клиента в ВСП
115,2321,Платеж с карты через ВСП


,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
1319032,30976504,26 00:00:00,5541,1110,-1864.11,NaN
6504404,68604501,146 19:35:20,6010,7030,33688.74,NaN
2080495,76924312,36 19:44:32,5411,1210,-20389.10,NaN


,customer_id,gender
6565,1370532,1
367,79751608,1
7658,88505830,0


Задания 1–4 нужно выполнить без использования функции `merge` и аналогичных по назначению.


>Допускается для таблицы **transactions** использование не всех записей. При чтении файлов обратите внимание на разделители внутри каждого из файлов – они могут различаться!


## Задание 1

1. Для столбца `tr_type` датафрейма `transactions` выберите произвольные `1000` строк с помощью метода `sample()`.

2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец `tr_description` в датафрейме `tr_types`), в которой содержится подстрока `'POS'` или `'ATM'`.


1. Для столбца `tr_type` датафрейма `transactions` выберите произвольные `1000` строк с помощью метода `sample()`.

In [10]:
sample_df = transactions.sample(n=1000, random_state=42)


2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец `tr_description` в датафрейме `tr_types`), в которой содержится подстрока `'POS'` или `'ATM'`.

In [11]:
type_dict = dict(zip(tr_types['tr_type'], tr_types['tr_description']))

sample_df['tr_description'] = sample_df['tr_type'].map(type_dict)

mask = sample_df['tr_description'].str.contains('POS|ATM', case=False, na=False)
share = mask.mean()

print(f"Доля транзакций с POS или ATM: {share:.2%}")

Доля транзакций с POS или ATM: 37.90%


## Задание 2


1. Для столбца `tr_type` датафрейма `transactions` посчитайте частоту встречаемости всех типов транзакций `tr_type` в `transactions`.
2. Выведите датафрейм, содержащий топ-10 транзакций, отсортированных по убыванию частоты встречаемости. Датафрейм должен содержать столбцы `tr_type`, `tr_description` и столбец с частотой встречаемости.

1. Для столбца `tr_type` датафрейма `transactions` посчитайте частоту встречаемости всех типов транзакций `tr_type` в `transactions`.

In [12]:
freq = transactions['tr_type'].value_counts().reset_index()
freq.columns = ['tr_type', 'frequency']

2. Выведите датафрейм, содержащий топ-10 транзакций, отсортированных по убыванию частоты встречаемости. Датафрейм должен содержать столбцы `tr_type`, `tr_description` и столбец с частотой встречаемости.

In [14]:
type_dict = tr_types.set_index('tr_type')['tr_description']
freq['tr_description'] = freq['tr_type'].map(type_dict)

top10 = freq.head(10)

top10

,tr_type,frequency,tr_description
0,1010,108018,Покупка. POS ТУ СБ РФ
1,7070,91478,Перевод на карту (с карты) через Мобильный бан...
2,2010,73687,Выдача наличных в АТМ Сбербанк России
3,1110,70342,Покупка. POS ТУ Россия
4,1030,55765,Оплата услуги. Банкоматы СБ РФ
5,2370,24595,Списание с карты на карту по операции <перевод...
6,7030,14715,Перевод на карту (с карты) через АТМ (в предел...
7,7010,13517,Взнос наличных через АТМ (в своем тер.банке)
8,1100,7985,Покупка. ТУ Россия
9,1200,5860,Покупка. Зарубеж. ТУ


## Задание 3
1. В датафрейме `transactions` найдите клиента с максимальной суммой приходов на карту.
2. В датафрейме `transactions` найдите клиента с максимальной суммой расходов по карте.
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов.

1. В датафрейме `transactions` найдите клиента с максимальной суммой приходов на карту.


In [16]:
incoming = transactions[transactions['amount'] > 0]

incoming_sum = incoming.groupby('customer_id')['amount'].sum()

max_income_client = incoming_sum.idxmax()
max_income_value = incoming_sum.max()

print(f"Клиент с максимальной суммой приходов: {max_income_client}")
print(f"Сумма приходов: {max_income_value:.2f}")

Клиент с максимальной суммой приходов: 70780820
Сумма приходов: 1248114886.81


2. В датафрейме `transactions` найдите клиента с максимальной суммой расходов по карте.


In [17]:
outgoing = transactions[transactions['amount'] < 0]

outgoing_sum = outgoing.groupby('customer_id')['amount'].sum()

max_expense_client = outgoing_sum.idxmin()
max_expense_value = outgoing_sum.min()

print(f"Клиент с максимальной суммой расходов: {max_expense_client}")
print(f"Сумма расходов: {max_expense_value:.2f}")

Клиент с максимальной суммой расходов: 70780820
Сумма расходов: -1249952204.79


3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов.

In [18]:
income_of_expense_client = incoming_sum.get(max_expense_client, 0)
expense_of_income_client = outgoing_sum.get(max_income_client, 0)

diff = abs(max_income_value - abs(max_expense_value))

print(f"Модуль разницы между суммой расходов и приходов: {diff:.2f}")

Модуль разницы между суммой расходов и приходов: 1837317.98


## Задание 4
1. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций из топ-10 в Задании 2.
1. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций, совершенных клиентами из Задания 3.

1. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций из топ-10 в Задании 2.


In [20]:
top10_df = transactions[transactions['tr_type'].isin(top10['tr_type'])]

stats_top10 = (
    top10_df.groupby('tr_type')['amount']
    .agg(['mean', 'median'])
    .reset_index()
)

type_dict = tr_types.set_index('tr_type')['tr_description']
stats_top10['tr_description'] = stats_top10['tr_type'].map(type_dict)

stats_top10

,tr_type,mean,median,tr_description
0,1010,-22278.022855,-8021.735,Покупка. POS ТУ СБ РФ
1,1030,-5969.889351,-2245.920,Оплата услуги. Банкоматы СБ РФ
2,1100,-49913.707967,-11530.530,Покупка. ТУ Россия
3,1110,-33136.148325,-11507.510,Покупка. POS ТУ Россия
4,1200,-40074.697833,-7389.060,Покупка. Зарубеж. ТУ
5,2010,-147014.396922,-44918.320,Выдача наличных в АТМ Сбербанк России
6,2370,-250533.662794,-58393.810,Списание с карты на карту по операции <перевод...
7,7010,320467.714705,112295.790,Взнос наличных через АТМ (в своем тер.банке)
8,7030,78114.733072,11903.350,Перевод на карту (с карты) через АТМ (в предел...
9,7070,61306.330284,10465.970,Перевод на карту (с карты) через Мобильный бан...


2. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций, совершенных клиентами из Задания 3.

In [21]:
selected_clients = [max_income_client, max_expense_client]
clients_df = transactions[transactions['customer_id'].isin(selected_clients)]

stats_clients = (
    clients_df.groupby('tr_type')['amount']
    .agg(['mean', 'median'])
    .reset_index()
)

stats_clients['tr_description'] = stats_clients['tr_type'].map(type_dict)
stats_clients

,tr_type,mean,median,tr_description
0,2010,-2.941056e+06,-3368873.660,Выдача наличных в АТМ Сбербанк России
1,2330,-2.382398e+06,-2245915.770,Списание с карты по операции “перевода с карты...
2,2370,-2.218505e+06,-2245915.770,Списание с карты на карту по операции <перевод...
3,6110,1.756293e+04,10028.010,Возврат покупки. POS ТУ Россия
4,7020,1.465129e+04,10319.985,Взнос наличных через POS
5,7021,2.850067e+04,28500.670,Взнос наличных через POS
6,7030,1.406196e+04,8848.910,Перевод на карту (с карты) через АТМ (в предел...
7,7031,1.921036e+04,12992.620,Перевод на карту (с карты) через АТМ (из одног...
8,7034,1.430648e+04,14306.480,Перевод на карту/ с карты через АТМ (без взима...
9,7040,2.153833e+04,14396.320,Перевод на карту (с карты) через POS (в предел...


## Подготовка для заданий 5–8

>**Из заданий 5-8 нужно выполнить два любых на выбор.**

Соедините датафрейм `transactions` со всеми остальными таблицами (`tr_mcc_codes`, `tr_types`, `gender_train`). Причем объединение с таблицей `gender_train` необходимо выполнить с помощью `left join`, а с оставшимися датафреймами – через `inner`.
После получения общей таблицы `gender_train`, `tr_types` и `tr_mcc_codes` можно удалить. В результате соединения датафреймов должно получиться `999584` строки.

In [70]:
tr_mcc_codes = tr_mcc_codes.drop_duplicates(subset=['mcc_code'])
tr_types = tr_types.drop_duplicates(subset=['tr_type'])

df = (
    transactions
    .merge(tr_mcc_codes, on='mcc_code', how='inner')
    .merge(tr_types, on='tr_type', how='inner')
    .merge(gender_train, on='customer_id', how='left')
)

print(df.shape)
df

(6846580, 9)


,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,mcc_description,tr_description,gender
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,1.0
1,39026145,1 10:19:29,6011,7010,56147.89,NaN,Финансовые институты — снятие наличности автом...,Взнос наличных через АТМ (в своем тер.банке),1.0
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN,Денежные переводы,Списание с карты по операции “перевода с карты...,1.0
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN,"Различные продовольственные магазины — рынки, ...",Покупка. POS ТУ СБ РФ,1.0
4,39026145,2 15:33:42,5499,1010,-920.83,NaN,"Различные продовольственные магазины — рынки, ...",Покупка. POS ТУ СБ РФ,1.0
...,...,...,...,...,...,...,...,...,...
6846575,61870738,453 16:03:02,5499,1010,-5176.84,10217113,"Различные продовольственные магазины — рынки, ...",Покупка. POS ТУ СБ РФ,0.0
6846576,61870738,454 10:54:60,5411,1010,-1652.77,022915,"Бакалейные магазины, супермаркеты",Покупка. POS ТУ СБ РФ,0.0
6846577,61870738,454 14:23:59,5499,1010,-4687.23,10217113,"Различные продовольственные магазины — рынки, ...",Покупка. POS ТУ СБ РФ,0.0
6846578,61870738,454 16:11:53,5541,1110,-4491.83,RU570124,Станции техобслуживания,Покупка. POS ТУ Россия,0.0


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата – отрицательное значение в столбце `amount`).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин.

>Обратите внимание, что для вычисления модуля разности не требуется точных знаний о том,
каким значением в таблице обозначены мужчины, а каким – женщины.

In [67]:
spending = df[df['amount'] < 0].groupby('gender')['amount'].mean()
income = df[df['amount'] > 0].groupby('gender')['amount'].mean()

spending_diff = abs(spending.iloc[0] - spending.iloc[1])
income_diff = abs(income.iloc[0] - income.iloc[1])

print("Модуль разницы средних трат:", spending_diff)
print("Модуль разницы средних поступлений:", income_diff)

Модуль разницы средних трат: 24748.03831067319
Модуль разницы средних поступлений: 65084.17392010853


## Задание 6

1. Для каждого типа транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу `amount`) отдельно для мужчин и женщин (назовите ее `max_income`). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям `max_income`.
2. Среди типов транзакций, найденных в пункте 1, выделите те, которые встречаются одновременно и у мужчин, и у женщин.

In [73]:
income = df[df['amount'] > 0]

income_max = (
    income.groupby(['tr_type', 'gender'], as_index=False)['amount']
    .max()
    .rename(columns={'amount': 'max_income'})
)

male_lowest = (
    income_max[income_max['gender'] == 0]
    .nsmallest(10, 'max_income')
)

female_lowest = (
    income_max[income_max['gender'] == 1]
    .nsmallest(10, 'max_income')
)

type_desc = df[['tr_type', 'tr_description']].drop_duplicates()

male_lowest = male_lowest.merge(type_desc, on='tr_type', how='left')
female_lowest = female_lowest.merge(type_desc, on='tr_type', how='left')

print("10 типов транзакций с минимальным max_income у мужчин:\n")
print(male_lowest[['tr_type', 'tr_description', 'max_income']].to_string(index=False))

print("\n10 типов транзакций с минимальным max_income у женщин:\n")
print(female_lowest[['tr_type', 'tr_description', 'max_income']].to_string(index=False))


10 типов транзакций с минимальным max_income у мужчин:

 tr_type                                                           tr_description  max_income
    4011                Плата за получение наличных в АТМ Сбербанка (в других ТБ)       50.53
    4021          Плата за получение наличных через POS в Сбербанке (в других ТБ)      592.70
    4071                 Плата за перевод на карту (с карты) через Мобильный банк     1010.66
    4051 Плата за перевод на карту (с карты) через АТМ (из одного ТБ в другой ТБ)     1122.96
    4110                                Плата за получение наличных в АТМ. Россия     4469.37
    4100                                      Плата за получение наличных. Россия     5041.86
    4210                              Плата за получение наличных в АТМ. Зарубеж.     5246.91
    2011                            Выдача наличных в АТМ Сбербанка (в других ТБ)     6737.75
    8146                                                                      н/д    14823.04
    

In [74]:
common_types = set(male_lowest['tr_type']) & set(female_lowest['tr_type'])

print("\nТипы транзакций, встречающиеся одновременно у мужчин и женщин:\n")
print(common_types)

common_desc = type_desc[type_desc['tr_type'].isin(common_types)]
print("\nОписание общих типов транзакций:\n")
print(common_desc.to_string(index=False))



Типы транзакций, встречающиеся одновременно у мужчин и женщин:

{4100, 4071, 4110, 4210, 4051, 8146}

Описание общих типов транзакций:

 tr_type                                                           tr_description
    4051 Плата за перевод на карту (с карты) через АТМ (из одного ТБ в другой ТБ)
    4071                 Плата за перевод на карту (с карты) через Мобильный банк
    4110                                Плата за получение наличных в АТМ. Россия
    4210                              Плата за получение наличных в АТМ. Зарубеж.
    4100                                      Плата за получение наличных. Россия
    8146                                                                      н/д


## Задание 7

1. Найдите суммы затрат по каждой категории (`mcc`) для мужчин и для женщин.
2. Найдите топ-10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc-кодов).

In [ ]:
# PUT YOUR CODE HERE

## Задание 8

1. Из поля `tr_datetime` выделите час `tr_hour`, в который произошла транзакция, как первые 2 цифры до `":"`.
2. Посчитайте количество транзакций со значением в столбце `amount` строго меньше `0` в ночное время для мужчин и женщин. Ночное время – это промежуток с 00:00 по 06:00 часов.

In [ ]:
# PUT YOUR CODE HERE

# Порядок защиты работы

**Для защиты работы необходимо:**
1. Предоставить результаты выполнения 6 заданий в виде ссылки на Google Colab.
2. Выполнить дополнительное задание в процессе сдачи работы. Дополнительное задание представляет собой запрос к данным лабораторной работы, по сложности аналогичный тем, что требовалось выполнить в рамках работы.

**Примеры дополнительных заданий:**
1. Найдите категорию транзакций, в которой совершается больше всего трат в ночное время (промежуток с 00:00 по 06:00). Выведите название категории и среднюю сумму трат в ней.
2. Найдите 3 категории транзакций, в которых средние траты мужчин и женщин различаются меньше всего.
3. Найдите терминал, через который проходит наибольшее число транзакций. Выведите идентификаторы 5 пользователей с наибольшей суммой отрицательных транзакций в этом терминале.

# Примечание

Рекомендуется выполнение заданий двумя способами – с помощью `pandas` и с помощью `SQL` с использованием модуля `sqlite3`.

# Дополнительные материалы

1. Pandas. User Guide [Электронный ресурс]. URL: https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html (дата обращения: 14.03.2024).
2. МакКинни, У. Python и анализ данных [Текст] / У. МакКинни ; пер. с англ. Слинкиной А. А. — Москва : ДМК-Пресс, 2023. — 536 с.
3. Хейдт, М. Изучаем pandas [Текст] / М. Хейдт. — Москва : ДМК Пресс, 2019. — 700 с.